In [28]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
import langchain_community.chat_models
from treetrace import TracingNode
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
import langchain

dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [6]:
factory = Factory.from_yaml("audience.yaml")

In [49]:
marie = factory.create_persona("Maria-gpt35")
lilly = factory.create_persona("Lilly-gpt35")

In [50]:
marie.actor

<OneShotLLMActor Maria>

In [40]:
marie.public_description()

'Public information about Maria: age 22\nMaria is a mentor\n'

In [54]:
link([marie, lilly])

In [66]:
with TracingNode("root") as root:
    ChatChannel.query_and_send_message(marie, lilly)
root.display()

In [35]:

class Situation:

    def __init__(self, subject, factory, person_keys):
        self.factory = factory
        self.person_keys = person_keys
        self.players = [self.factory.create_persona(key) for key in person_keys]
        names = ",".join(p.name for p in self.players)
        judge_model = langchain_community.chat_models.ChatOpenAI(model="gpt-3.5-turbo")
        self.judge = interlab.actor.llm_actor.OneShotLLMActor("Judge", judge_model, "You observe the following communication:")
        self.subject = subject

        @dataclass
        class JudgeAction:
            has_agreement: int = Field(
                description=f"1 if and only if all sides ({names}) made an agreement on {subject}. Otherwise 0",
            )
            agreement_result: str | None = Field(
                description=f"{self.subject} that parties agreed on ({names}); otherwise null",
            )
        self.judge_action = JudgeAction

    def agreement_check(self):
        players = situation.players
        self.judge.query()

    

def resolve_direct(sitation, channel=ChatChannel, max_steps_per_player=7):
    players = situation.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [situation.judge])
        if situation.agreement_check():
            return

with TracingNode("root") as root:
    situation = Situation("the same movie", factory, ["Maria-gpt35", "Lilly-gpt35"])
    resolve_direct(situation)
root.display()

KeyboardInterrupt: 

In [36]:
root.display()